# Scratch code for testing purposes and small experiments. 


In [40]:

import numpy as np
import importlib
from Graph import Graph
from helpers import *
import pandas as pd
import random
from tqdm import tqdm
from deterministic_attack import DeterministicAttack
from revisited_spectral import RevisitedSpectral


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Block matrix generation

In [8]:

block = Graph.block_from_txt("datasets/block.txt")

p_flip = 0.1

for i in range(0, block.first_partition_size):
    for j in range(block.first_partition_size+1, block.adj_matrix.shape[1]):
        if block.adj_matrix[i, j] == 1 and np.random.rand() < p_flip:
                block.adj_matrix[i, j] = not block.adj_matrix[i, j]
                block.adj_matrix[j, i] = not block.adj_matrix[j, i]

        elif block.adj_matrix[i, j] == 0 and np.random.rand() < p_flip:
                block.adj_matrix[i, j] = not block.adj_matrix[i, j]
                block.adj_matrix[j, i] = not block.adj_matrix[j, i]

block.to_txt(f"datasets/block_{p_flip}.txt")




## Exploring the spectral approach

In [61]:
dataset = "netscience"
G = Graph.from_txt(f"datasets/{dataset}.txt")
G1, G2 = G.split_dataset(common_prop=0.0, graph1_prop=0.0)
A = np.dot(G.adj_matrix, G.adj_matrix.T)

In [66]:
deterministic_ttack = DeterministicAttack(G1, A)
deterministic_ttack.run()
reconstructed_graph = deterministic_ttack.get_reconstructed_graph()

probabilistic_attack = RevisitedSpectral(reconstructed_graph, A)
probabilistic_attack.run(alpha=1, beta=1, threshold=0.5)
reconstructed_graph = probabilistic_attack.get_reconstructed_graph()
probabilistic_attack.sanity_check()
reconstructed_graph = probabilistic_attack.get_reconstructed_graph()

deterministic_ttack = DeterministicAttack(reconstructed_graph, A)
deterministic_ttack.run()
reconstructed_graph = deterministic_ttack.get_reconstructed_graph()
reconstructed_graph.fix_edges()
print("Errors: ", np.sum(np.abs(reconstructed_graph.adj_matrix - G.adj_matrix)))




Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 17491.65it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  |  3051 |
| Number of reconstructed edges |   90  |
|    Number of unknown edges    | 68869 |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 18118.81it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 22548 |
| Number of reconstructed edges |   90  |
|    Number of unknown edges    | 49372 |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 17258.14it/s]


...

+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71081 |
| Number of reconstructed edges |  905  |
|    Number of unknown edges    |   24  |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 908/908 [00:00<00:00, 18009.48it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71084 |
| Number of reconstructed edges |  908  |
|    Number of unknown edges    |   18  |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 908/908 [00:00<00:00, 30084.27it/s]

+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71084 |
| Number of reconstructed edges |  908  |
|    Number of unknown edges    |   18  |
+-------------------------------+-------+ 

Errors:  60


### Sign assignment

### Revisited sign assignment

# Erdos et al.

In [63]:
reconstructed_graph = erdos(A, threshold=0.5)
print("Errors: ", np.sum(np.abs(reconstructed_graph - G.adj_matrix)))

NameError: name 'erdos' is not defined

# Deterministic - Probabilistic

In [23]:
reconstructed_graph = graph1_copy(G1)
reconstructed_graph, number_modifs_degree = degree_attack(reconstructed_graph, A , [1, 2])
reconstructed_graph = deterministics(reconstructed_graph, A)
reconstructed_graph = Graph.from_adj_matrix(GRAND(A, reconstructed_graph, 0.5, 1, len(G.edges()) / 1000))
print("Errors: ", np.sum(np.abs(reconstructed_graph.adj_matrix - G.adj_matrix)))

Rectangle attack:: 100%|██████████| 200/200 [00:00<00:00, 18124.60it/s]
Triangle attack: 0it [00:00, ?it/s]


+-------------------------------+---------+
|              Stat             |  Value  |
+-------------------------------+---------+
|   Number of impossible edges  |   3025  |
| Number of reconstructed edges |    0    |
|    Number of unknown edges    | 4573800 |
+-------------------------------+---------+ 



GRAND: 100%|██████████| 3025/3025 [5:10:02<00:00,  6.15s/it]      


Errors:  345


# Deterministic - Probabilistic - Deterministic

In [24]:
reconstructed_graph = graph1_copy(G1)
reconstructed_graph, number_modifs_degree = degree_attack(reconstructed_graph, A , [1, 2])
for i in range(1):
    nodes_to_forget = set()
    links = set()
    reconstructed_graph = deterministics(reconstructed_graph, A)
    last_deter = reconstructed_graph.copy()
    r = reconstructed_graph.copy()
    reconstructed_graph = Graph.from_adj_matrix(GRAND(A, reconstructed_graph, 0.5, 1, 1))
    print("Errors: ", np.sum(np.abs(reconstructed_graph.adj_matrix - G.adj_matrix)))

    A_prime = np.dot(reconstructed_graph.adj_matrix, reconstructed_graph.adj_matrix.T)
    for i in range(A.shape[0]):
        for j in range(i, A.shape[1]):
            if A[i, j] != A_prime[i, j] and A[i, i] == A_prime[i, i]:
                for k in range(A.shape[0]):
                    reconstructed_graph.adj_matrix[j, k] = 2
                    reconstructed_graph.adj_matrix[k, j] = 2
                    nodes_to_forget.add(j)
            if A[i, j] != A_prime[i, j] and A[j, j] == A_prime[j, j]:
                for k in range(A.shape[0]):
                    reconstructed_graph.adj_matrix[i, k] = 2
                    reconstructed_graph.adj_matrix[k, i] = 2
                    nodes_to_forget.add(i)

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if last_deter.adj_matrix[i, j] != 2:
                reconstructed_graph.adj_matrix[i, j] = last_deter.adj_matrix[i, j]


reconstructed_graph = deterministics(reconstructed_graph, A)

reconstructed_graph.fix_edges()
print("Errors: ", np.sum(np.abs(reconstructed_graph.adj_matrix - G.adj_matrix)))


Rectangle attack:: 100%|██████████| 200/200 [00:00<00:00, 32388.45it/s]
Triangle attack: 0it [00:00, ?it/s]


+-------------------------------+---------+
|              Stat             |  Value  |
+-------------------------------+---------+
|   Number of impossible edges  |   3025  |
| Number of reconstructed edges |    0    |
|    Number of unknown edges    | 4573800 |
+-------------------------------+---------+ 



GRAND: 100%|██████████| 3025/3025 [10:08<00:00,  4.97it/s]


Errors:  352


Triangle attack: 100%|██████████| 13002/13002 [00:00<00:00, 15189.40it/s]


+-------------------------------+---------+
|              Stat             |  Value  |
+-------------------------------+---------+
|   Number of impossible edges  | 4549882 |
| Number of reconstructed edges |  13002  |
|    Number of unknown edges    |  13941  |
+-------------------------------+---------+ 



Triangle attack: 100%|██████████| 13002/13002 [00:00<00:00, 15247.48it/s]


+-------------------------------+---------+
|              Stat             |  Value  |
+-------------------------------+---------+
|   Number of impossible edges  | 4553002 |
| Number of reconstructed edges |  13002  |
|    Number of unknown edges    |  10821  |
+-------------------------------+---------+ 



Triangle attack: 100%|██████████| 13002/13002 [00:00<00:00, 15150.49it/s]


+-------------------------------+---------+
|              Stat             |  Value  |
+-------------------------------+---------+
|   Number of impossible edges  | 4553002 |
| Number of reconstructed edges |  13002  |
|    Number of unknown edges    |  10821  |
+-------------------------------+---------+ 

Errors:  348


# Experiments on graphs with the same G^2

In [ ]:
import pandas as pd

g1 = pd.read_csv("datasets/josee.txt", sep="\t", header=None)
 

g2_matrix = np.array([[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
                    [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
                    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
                    [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
                    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
                    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
                    [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                    [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
                    ])


g1_matrix = Graph.from_txt("datasets/josee.txt").adj_matrix
# perm_1 = [0, 2, 4, 6, 8, 10, 1, 3, 5, 7, 9, 11]
# perm_2 = [1, 3, 5, 7, 9, 11, 0, 2, 4, 6, 8, 10]

g1_matrix_perm = g1_matrix[perm_1, :][:, perm_1]
g2_matrix_perm = g2_matrix[perm_2, :][:, perm_2]


G_square_1 = np.dot(g1_matrix, g1_matrix)
G_square_2 = np.dot(g2_matrix, g2_matrix)
# print(G_square_1)
# print(G_square_2)




rec = erdos(G_square, 0.5)
rec_perm1 = rec[perm_1, :][:, perm_1]

rec_pol = erdos(A, 0.5)

print("Errors G : ", np.linalg.norm(rec_pol - G.adj_matrix) ** 2)

